In [ ]:
import os
os.chdir("..")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sb
import math

In [ ]:
from project.kerneldensityestimation import KernelDensityEstimation as KDE
from project.mixturedensity import HomogeneousMixtureDensity as MixDens
from project.model import OtherModel as Model
from project.dataholder import DataHolder
from project.lossfunction import MaximizeLikelihood
from project.optimizer import SPSA, SA
from project.topographicmap import TopographicMap

In [ ]:
datafolder = "data/"
imagefolder = "figures/homomixture/"

In [ ]:
df = pd.read_csv(datafolder+"data.tsv", delimiter="\t")

In [ ]:
df

In [ ]:
n_bins = 2
continue_flag = True
while continue_flag:
	counts,_,_ = np.histogram2d(df["rproj"], df["vproj"], bins=n_bins, range=[[0,10],[-3,3]])
	print(f"Using {n_bins} bins, with min {counts.min()}", end="\r", flush=True)
	if counts.min() < 50:
		continue_flag = False
		n_bins -= 1
		counts,_,_ = np.histogram2d(df["rproj"], df["vproj"], bins=n_bins, range=[[0,10],[-3,3]])
	else:
		n_bins += 1
print(f"Using {n_bins} bins, with min {counts.min()}", flush=True)

In [ ]:
sb.histplot(data=df, x="rproj", y="vproj", bins=n_bins, cbar=True)

In [ ]:
vbins = np.linspace(3, -3, n_bins+1)
rbins = np.linspace(0, 10, n_bins+1)
binwidth = 5
r3dbins = np.arange(0, df["# r3d"].max()+binwidth, binwidth)
xs = np.linspace(r3dbins.min(), r3dbins.max(), 101)

In [ ]:
n_kernels = 4
tm = TopographicMap(0, 10, -3, 3, n_kernels, n_kernels)

In [ ]:
mixture = MixDens()
n_gaussians = 10
gauss_centres = np.linspace(r3dbins.min(), r3dbins.max(), n_gaussians)
for g in gauss_centres:
	mixture.addGaussian(1/n_gaussians, g, 1)

# With transformed parameters

In [ ]:
model = Model(tm, mixture)
def pymixture(xs, a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, s0, s1, s2, s3, s4, s5, s6, s7, s8, s9):
	mixture = model.evalOnThetas(np.array([a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, s0, s1, s2, s3, s4, s5, s6, s7, s8, s9]))
	return mixture(xs)

args = (1,) * 10 + (2.8,) * 10
plt.plot(xs, pymixture(xs,*args))

In [ ]:
fig, axs = plt.subplots(n_bins, n_bins, figsize=(n_bins*4,)*2)

initial_params = []
n_evals_transformed = []
for i in range(n_bins):
	rmask = np.logical_and(df["rproj"]>=rbins[i], df["rproj"]<=rbins[i+1])
	for j in range(n_bins):
		vmask = np.logical_and(df["vproj"]>=vbins[j+1], df["vproj"]<=vbins[j])
		subset = df.loc[np.logical_and(rmask, vmask)]
		axs[j,i].hist(subset["# r3d"], bins=r3dbins, density=True, alpha=0.5, label="data")
		values = subset["# r3d"].to_numpy()
		kde = KDE(values, 3)
		xs = np.linspace(r3dbins.min(), r3dbins.max(), 101)
		axs[j,i].plot(xs, kde(xs), linewidth=5, label="kde")
		try:
			params,_,output_dict,_,_ = curve_fit(pymixture, xs, kde(xs), p0=[1]*10+[2.5]*10, bounds=([-np.inf]*10+[-10]*10, [10]*20), full_output=True, maxfev=10000)
			n_evals_transformed.append(output_dict["nfev"])
			mixture = model.evalOnThetas(params)
			axs[j,i].plot(xs, mixture(xs), linestyle="--", linewidth=5, label="mixture")
		except Exception as error:
			print(error)
			params = [np.nan]*20
		initial_params.append(params)
		if j == n_bins - 1:
			axs[j,i].set_xlabel(r"$r_\mathrm{3D}$")
		else:
			axs[j,i].set_xticks([])
		if i == 0:
			axs[j,i].set_ylabel(r"$P(r_\mathrm{3D})$")
		else:
			axs[j,i].set_yticks([])
		axs[j,i].set_ylim([0,0.05])
		axs[j,i].legend(title=f"r={rbins[i:i+2].mean():.1f}, v={vbins[j:j+2].mean():.1f}")

fig.tight_layout()
fig.savefig(imagefolder+"initialization-with-transformed.png")

initial_params_transformed = np.array(initial_params).T

In [ ]:
coeffs = initial_params_transformed[:10] / initial_params_transformed[:10].sum(axis=0)

In [ ]:
coeffs.shape

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(4,20))

for i in range(10):
	for j in range(2):
		if j == 0:
			plottingdata = coeffs[i].reshape((15,15)).T
			sb.heatmap(data=plottingdata, vmin=0, vmax=1, ax=axs[i,j])
			axs[i,j].set_title(f"coeff {i}")
		if j == 1:
			plottingdata = initial_params_transformed[10+i].reshape((15,15)).T
			sb.heatmap(data=plottingdata, ax=axs[i,j])
			axs[i,j].set_title(f"sigma {i}")
fig.tight_layout()
fig.savefig(imagefolder+"parameters-with-transformed.png")

# With raw parameters

In [ ]:
def pymixture(xs, a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, s0, s1, s2, s3, s4, s5, s6, s7, s8, s9):
	mixture = MixDens()
	for g in gauss_centres:
		mixture.addGaussian(1/n_gaussians, g, 1)
	params = np.array([a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, s0, s1, s2, s3, s4, s5, s6, s7, s8, s9])
	mixture.setParams(params)
	return mixture(xs)
	
plt.plot(xs, pymixture(xs,*args))

In [ ]:
fig, axs = plt.subplots(n_bins, n_bins, figsize=(n_bins*4,)*2)

initial_params = []
n_evals_raw = []
for i in range(n_bins):
	rmask = np.logical_and(df["rproj"]>=rbins[i], df["rproj"]<=rbins[i+1])
	for j in range(n_bins):
		vmask = np.logical_and(df["vproj"]>=vbins[j+1], df["vproj"]<=vbins[j])
		subset = df.loc[np.logical_and(rmask, vmask)]
		axs[j,i].hist(subset["# r3d"], bins=r3dbins, density=True, alpha=0.5, label="data")
		values = subset["# r3d"].to_numpy()
		kde = KDE(values, 3)
		xs = np.linspace(r3dbins.min(), r3dbins.max(), 101)
		axs[j,i].plot(xs, kde(xs), linewidth=5, label="kde")
		try:
			params,_,output_dict,_,_ = curve_fit(pymixture, xs, kde(xs), p0=[1]*10+[2.5]*10, bounds=([0]*20, [1]*10+[10]*10), full_output=True, maxfev=10000)
			#params,_,output_dict,_,_ = curve_fit(pymixture, xs, kde(xs), p0=[1,1,1,10,60,10,90,10], bounds=([-np.inf,-np.inf,-np.inf,-np.inf,-np.inf,-np.inf,-np.inf,-np.inf,-np.inf],[np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf]), full_output=True)
			n_evals_raw.append(output_dict["nfev"])
			mixture = MixDens()
			for g in gauss_centres:
				mixture.addGaussian(1, g, 1)
			mixture.setParams(params)
			axs[j,i].plot(xs, mixture(xs), linestyle="--", linewidth=5, label="mixture")
		except Exception as error:
			print(error)
			params = [np.nan]*20
		initial_params.append(params)
		if j == n_bins - 1:
			axs[j,i].set_xlabel(r"$r_\mathrm{3D}$")
		else:
			axs[j,i].set_xticks([])
		if i == 0:
			axs[j,i].set_ylabel(r"$P(r_\mathrm{3D})$")
		else:
			axs[j,i].set_yticks([])
		axs[j,i].set_ylim([0,0.05])
		axs[j,i].legend(title=f"r={rbins[i:i+2].mean():.1f}, v={vbins[j:j+2].mean():.1f}")

fig.tight_layout()
fig.savefig(imagefolder+"initialization-with-raw.png")

initial_params_raw = np.array(initial_params).T

In [ ]:
coeffs = initial_params_raw[:10] / initial_params_raw[:10].sum(axis=0)

In [ ]:
coeffs.shape

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(4,20))

for i in range(10):
	for j in range(2):
		if j == 0:
			plottingdata = coeffs[i].reshape((15,15)).T
			sb.heatmap(data=plottingdata, vmin=0, vmax=1, ax=axs[i,j])
			axs[i,j].set_title(f"coeff {i}")
		if j == 1:
			plottingdata = initial_params_raw[10+i].reshape((15,15)).T
			sb.heatmap(data=plottingdata, ax=axs[i,j])
			axs[i,j].set_title(f"sigma {i}")
fig.tight_layout()
fig.savefig(imagefolder+"parameters-with-raw.png")

In [ ]:
colors = {"trans":"blue", "raw":"red"}
plt.plot(n_evals_transformed, color=colors["trans"], label="transformed", alpha=0.5)
plt.plot(n_evals_raw, color=colors["raw"], label="raw", alpha=0.5)
plt.axhline(np.mean(n_evals_transformed), color=colors["trans"], linestyle=":")
plt.axhline(np.mean(n_evals_raw), color=colors["raw"], linestyle=":")
plt.plot([], [], linestyle=":", color="grey", label="mean")
plt.legend()
plt.ylim(bottom=0)
plt.savefig(imagefolder+"funceval-comparison.png")